In [1]:
import pandas as pd
import torch
from torch import nn
import gensim
from gensim.models import KeyedVectors
import sklearn
import torchmetrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('main_dataset.csv',  encoding='UTF-8')

In [3]:
df.head()

,id,text,label
0,1,на работе был полный пиддес :| и так каждое за...,0.0
1,2,"Коллеги сидят рубятся в Urban terror, а я из-з...",1.0
2,3,@elina_4post как говорят обещаного три года жд...,0.0
3,4,"Желаю хорошего полёта и удачной посадки,я буду...",0.0
4,5,"Обновил за каким-то лешим surf, теперь не рабо...",0.0


In [4]:
df.isna().sum()

id       0
text     0
label    0
dtype: int64

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [6]:
after = after.dropna(subset=['text'])


NameError: name 'after' is not defined

In [ ]:
train_df, test_df = train_test_split(after, test_size=0.3, random_state=42)

In [ ]:
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix


#Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

#Преобразование текстов в матрицу TF-IDF
tfidf_matrix_train_df = vectorizer.fit_transform(train_df['text'])

#Преобразование разреженной матрицы в формат CSR
tfidf_csr_train_df = csr_matrix(tfidf_matrix_train_df)

# Преобразование формата разреженной матрицы в формат COO
tfidf_coo_train_df = tfidf_csr_train_df.tocoo().astype(np.float32)

# Преобразование матрицы TF-IDF в разреженный тензор PyTorch
tfidf_tensor_train_df = torch.sparse.FloatTensor(torch.LongTensor([tfidf_coo_train_df.row.tolist(), tfidf_coo_train_df.col.tolist()]),
                                        torch.FloatTensor(tfidf_coo_train_df.data))

# Вывод тензора TF-IDF
print(tfidf_tensor_train_df)

In [ ]:
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

#Удаление строк с пропущенными значениями в столбце 'text'
after = after.dropna(subset=['text'])

#Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

#Преобразование текстов в матрицу TF-IDF
tfidf_matrix_test_df = vectorizer.fit_transform(test_df['text'])

#Преобразование разреженной матрицы в формат CSR
tfidf_csr_test_df = csr_matrix(tfidf_matrix_test_df)

# Преобразование формата разреженной матрицы в формат COO
tfidf_coo_test_df = tfidf_csr_test_df.tocoo().astype(np.float32)

# Преобразование матрицы TF-IDF в разреженный тензор PyTorch
tfidf_tensor_test_df = torch.sparse.FloatTensor(torch.LongTensor([tfidf_coo_test_df.row.tolist(), tfidf_coo_test_df.col.tolist()]),
                                        torch.FloatTensor(tfidf_coo_test_df.data))

# Вывод тензора TF-IDF
print(tfidf_tensor_test_df)

In [ ]:
y_train_list=list()
for i in train_df['label']:
    y_train_list.append(i)
y_train=torch.tensor(y_train_list)
y_train.unsqueeze_(1)

In [ ]:
y_train_list=list()
for i in train_df['label']:
    y_train_list.append(i)
y_train=torch.tensor(y_train_list)
y_train.unsqueeze_(1)

In [ ]:
x_train = tfidf_tensor_train_df
x_test = tfidf_tensor_test_df 

In [ ]:
x_train = x_train.to_dense()
x_test = x_test.to_dense()
x_train = x_train.narrow(1, 0, 1000) 
x_test = x_test.narrow(1, 0, 1000) 

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#1 (Base)</font>

In [ ]:
#----------------------------------------------

In [ ]:
class MLP_MK1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_MK1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
#         out=torch.round(out)
        return out

In [ ]:
Mk1=MLP_MK1(1000,300,1)

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk1=Mk1.to(dev)

In [ ]:
optimizer = torch.optim.Adam(Mk1.parameters(), lr=0.001)
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy'])
for epoch in range(10000):
    optimizer.zero_grad()
    pred = Mk1.forward(x_train.float())
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf = pd.concat([diagdf, pd.DataFrame(data, index=[epoch])], ignore_index=True)
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# построение графика loss
plt.plot(diagdf['Epoch'], diagdf['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf['Epoch'], diagdf['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf['Epoch'], diagdf['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test1=Mk1.forward(x_test.float())
pred_test_binarized1 = (pred_test1 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized1.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#2</font>

<font color='red'>epochs: 10000 -> 5050, hidden_size: 200 -> 300, lr: 0.0001 -> 0.001</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK2(nn.Module): #изменить название
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_MK2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size) #если добавлять или изменять, то не забыть поменять название
        
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
Mk2=MLP_MK2(1000,300,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk2=Mk2.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk2.parameters(), lr=0.001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf1=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(5050):
    optimizer.zero_grad()
    pred = Mk2.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf1 = pd.concat([diagdf1, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf1['Epoch'], diagdf1['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf1['Epoch'], diagdf1['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf1['Epoch'], diagdf1['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test2=Mk2.forward(x_test.float())
pred_test_binarized2 = (pred_test2 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized2.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#3</font>

<font color='red'>layers + 1, epochs: 5050 -> 2000, hidden_size: 300 -> 150, lr: 0.001</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK3(nn.Module): #изменить название
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_MK3, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.fc3 = nn.Linear(hidden_size, output_size) #если добавлять или изменять, то не забыть поменять название
        self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = self.fc3(out)
        out = self.tanh(out)
        return out

In [ ]:
Mk3=MLP_MK3(1000,150,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk3=Mk3.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk3.parameters(), lr=0.001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf2=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(2000):
    optimizer.zero_grad()
    pred = Mk3.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf2 = pd.concat([diagdf2, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf2['Epoch'], diagdf2['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf2['Epoch'], diagdf2['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf2['Epoch'], diagdf2['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test3=Mk3.forward(x_test.float())
pred_test_binarized3 = (pred_test3 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized3.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#4</font>

<font color='red'>epochs: 2000 -> 5000, lr: 0.001 -> 0.0001</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK4(nn.Module): #изменить название
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_MK4, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.fc3 = nn.Linear(hidden_size, output_size) #если добавлять или изменять, то не забыть поменять название
        self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = self.fc3(out)
        out = self.tanh(out)
        return out

In [ ]:
Mk4=MLP_MK4(1000,150,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk4=Mk4.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk4.parameters(), lr=0.0001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf3=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(5000):
    optimizer.zero_grad()
    pred = Mk4.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf3 = pd.concat([diagdf3, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf3['Epoch'], diagdf3['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf3['Epoch'], diagdf3['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf3['Epoch'], diagdf3['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test4=Mk4.forward(x_test.float())
pred_test_binarized4 = (pred_test4 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized4.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#5</font>

<font color='red'>layers: -1, activation functions: Sigmoid() and Tanh(), epochs: 2000, hidden_size: 200, lr: 0.0001 -> 0.001</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK5(nn.Module): #изменить название
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_MK5, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.sigmoid1 = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size, output_size) #если добавлять или изменять, то не забыть поменять название
        self.tanh1 = nn.Tanh()
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
        out = self.sigmoid1(out)
        out = self.fc2(out)
        out = self.tanh1(out)
        return out

In [ ]:
Mk5=MLP_MK5(1000,200,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk5=Mk5.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk5.parameters(), lr=0.001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf4=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(2000):
    optimizer.zero_grad()
    pred = Mk5.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf4 = pd.concat([diagdf4, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf4['Epoch'], diagdf4['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf4['Epoch'], diagdf4['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf4['Epoch'], diagdf4['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test5=Mk5.forward(x_test.float())
pred_test_binarized5 = (pred_test5 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized5.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#6</font>

<font color='red'>layer + 1, epochs: 2000 -> 10000, lr: 0.0001 -> 0.001, hidden_size: 200 -> 300</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK6(nn.Module): #изменить название
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_MK6, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.fc3 = nn.Linear(hidden_size, output_size) #если добавлять или изменять, то не забыть поменять название
        self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = self.fc3(out)
        out = self.tanh(out)
        return out

In [ ]:
Mk6=MLP_MK6(1000,300,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk6=Mk6.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk6.parameters(), lr=0.001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf5=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(10000):
    optimizer.zero_grad()
    pred = Mk6.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf5 = pd.concat([diagdf5, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf5['Epoch'], diagdf5['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf5['Epoch'], diagdf5['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf5['Epoch'], diagdf5['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test6=Mk6.forward(x_test.float())
pred_test_binarized6 = (pred_test6 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized6.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#7</font>

<font color='red'>layers - 2, epochs: 10000, lr: 0.001</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK7(nn.Module): #изменить название
    def __init__(self, input_size, output_size):
        super(MLP_MK7, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
#         self.fc3 = nn.Linear(hidden_size, output_size) #если добавлять или изменять, то не забыть поменять название
#         self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
#         out = self.relu(out)
#         out = self.fc2(out)
        out = self.sigmoid(out)
#         out = self.fc3(out)
#         out = self.tanh(out)
        return out

In [ ]:
Mk7=MLP_MK7(1000,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk7=Mk7.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk7.parameters(), lr=0.001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf6=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(10000):
    optimizer.zero_grad()
    pred = Mk7.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf6 = pd.concat([diagdf6, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf6['Epoch'], diagdf6['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf6['Epoch'], diagdf6['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf6['Epoch'], diagdf6['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test7=Mk7.forward(x_test.float())
pred_test_binarized7 = (pred_test7 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized7.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#8</font>

<font color='red'>layers: 1, epochs: 20000, lr: 0.001 -> 0.0001</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK8(nn.Module): #изменить название
    def __init__(self, input_size, output_size):
        super(MLP_MK8, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
#         self.fc3 = nn.Linear(hidden_size, output_size) #если добавлять или изменять, то не забыть поменять название
#         self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
#         out = self.relu(out)
#         out = self.fc2(out)
        out = self.sigmoid(out)
#         out = self.fc3(out)
#         out = self.tanh(out)
        return out

In [ ]:
Mk8=MLP_MK8(1000,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk8=Mk8.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk8.parameters(), lr=0.0001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf7=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(20000):
    optimizer.zero_grad()
    pred = Mk8.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf7 = pd.concat([diagdf7, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf7['Epoch'], diagdf7['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf7['Epoch'], diagdf7['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf7['Epoch'], diagdf7['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test8=Mk8.forward(x_test.float())
pred_test_binarized8 = (pred_test8 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized8.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#9</font>

<font color='red'>layers: 1, epochs: 20000, lr: 0.0001 -> 0.001</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK9(nn.Module): #изменить название
    def __init__(self, input_size, output_size):
        super(MLP_MK9, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
#         self.fc3 = nn.Linear(hidden_size, output_size) #если добавлять или изменять, то не забыть поменять название
#         self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
#         out = self.relu(out)
#         out = self.fc2(out)
        out = self.sigmoid(out)
#         out = self.fc3(out)
#         out = self.tanh(out)
        return out

In [ ]:
Mk9=MLP_MK9(1000,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk9=Mk9.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk9.parameters(), lr=0.001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf8=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(20000):
    optimizer.zero_grad()
    pred = Mk9.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf8 = pd.concat([diagdf8, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf8['Epoch'], diagdf8['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf8['Epoch'], diagdf8['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf8['Epoch'], diagdf8['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test9=Mk9.forward(x_test.float())
pred_test_binarized9 = (pred_test9 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized9.to('cpu')))

In [ ]:
#----------------------------------------------

<font color='red'>Word2Vec_MLP_Experiment_#10</font>

<font color='red'>layers: 1+3, epochs: 10000, lr: 0.0001</font>

In [ ]:
#----------------------------------------------

In [ ]:
# import torch.nn as nn

# nn.Linear() можно указать точные значения для "input_size, hidden_size"
# вместо nn.ReLU() можно посмотреть Sigmoid, tangensoid, etc.
# попробовать добавить еще Layers (fc3, fc4, etc.)
class MLP_MK10(nn.Module): #изменить название
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_MK10, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 200)
        self.sigmoid = nn.Sigmoid()
        self.fc3 = nn.Linear(200, 100) #если добавлять или изменять, то не забыть поменять название
        self.tanh = nn.Tanh()
        self.fc4 = nn.Linear(100, output_size) #если добавлять или изменять, то не забыть поменять название
        self.sigmoid1 = nn.Sigmoid()
    def forward(self, x):
        out = self.fc1(x) #если добавлять или изменять, то не забыть поменять название
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid1(out)
        return out

In [ ]:
Mk10=MLP_MK10(1000,300,1) #изменить название переменной

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
else:
    dev = torch.device("cpu")

In [ ]:
# if torch.cuda.is_available():
#     dev=torch.device('cuda:0')
# else: dev=torch.device('cpu:0')
torch.cuda.is_available()

In [ ]:
x_train=x_train.to(dev)
x_test=x_test.to(dev)
y_train=y_train.to(dev)
y_test=y_test.to(dev)

In [ ]:
Mk10=Mk10.to(dev)

In [ ]:
# "lr" - learning rate (чем больше "epoch", тем меньше "lr")
optimizer = torch.optim.Adam(Mk10.parameters(), lr=0.0001) #изменить название переменной
loss = nn.BCEWithLogitsLoss()
f1_score = torchmetrics.functional.f1_score
accuracy = torchmetrics.functional.accuracy
diagdf9=pd.DataFrame(columns=['Epoch','loss','F1-score','accuracy']) #изменить название переменной
# можно изменить количество "epoch"
for epoch in range(10000):
    optimizer.zero_grad()
    pred = Mk10.forward(x_train.float()) #изменить название переменной
    l = loss(pred, y_train) # squeeze the predicted tensor
    l.backward()
    f1= f1_score(pred, y_train, task='binary')
    acc=accuracy(pred, y_train, task='binary')
    data={'Epoch':epoch,'loss':l.item(),'F1-score':f1.item(),'accuracy':acc.item()}
    diagdf9 = pd.concat([diagdf9, pd.DataFrame(data, index=[epoch])], ignore_index=True) #изменить название переменной
    optimizer.step()
    print('Epoch: ', epoch+1, ', Loss: ', l.item(),'  f1-score:', f1.item(),"  accuracy:", acc.item())

In [ ]:
# diagdf изменить название везде
# построение графика loss
plt.plot(diagdf9['Epoch'], diagdf9['loss'], label='loss')

# построение графика F1-score
plt.plot(diagdf9['Epoch'], diagdf9['F1-score'], label='F1-score')

# построение графика accuracy
plt.plot(diagdf9['Epoch'], diagdf9['accuracy'], label='accuracy')

# добавление заголовка графика
plt.title('Графики loss, F1-score и accuracy')

# добавление подписей к осям
plt.xlabel('Epoch')
plt.ylabel('Значение')

# добавление легенды
plt.legend()

# отображение графика
plt.show()

In [ ]:
pred_test10=Mk10.forward(x_test.float())
pred_test_binarized10 = (pred_test10 > 0.5).float()

In [ ]:
print(classification_report(y_test.to('cpu'), pred_test_binarized10.to('cpu')))